In [1]:
import random
#from itertools import chain, combinations #powerset için importladık

class Item(object): #bas ve son sayfa için olusturulan Item classımız
    def __init__(self,basS,sonS):
        
        self.basSayfa=basS
        self.sonSayfa=sonS
    def getbasSayfa(self):
        return self.basSayfa
    def getsonSayfa(self):
        return self.sonSayfa
    def __str__(self): #print işleminin böyle yapıldığını belirtiyor.
        result= '<'+str(self.basSayfa)+', '+str(self.sonSayfa)+ '>'
        return result


In [2]:
def randList(kisiS,sayfaS): #kisi sayisi ve sayfa sayısına göre random listemizi olusturyoruz.
    Items=[]
    attachedList=[] #bu listenin amaci en cok okunan sayfayı bulmak için bütün elemanları bunda topluyoruz.

    for i in range(kisiS):  
       baslangicS=random.randint(1,sayfaS) 
       sonS=random.randint(baslangicS,sayfaS)  #baslangıc sayfasına göre random atıyoruz ve items'ımıza ekliyoruz
       Items.append(Item(baslangicS,sonS)) 
       while(sonS>=baslangicS): #bu döngünün amacı kisi sayısına göre baslangıc ve son sayfamıza göre bütün ara elemanlar dahil attachedlistemizde toplamak.
           attachedList.append(baslangicS)
           baslangicS+=1

    return Items,attachedList

In [3]:
def get_hist(my_list): #histogramimizi cikariyoruz 
    my_hist={}
    for w in my_list:
        if w in my_hist.keys():
            my_hist[w]=my_hist[w]+1
        else:
            my_hist[w]=1
    return my_hist


In [4]:
def mostReadPage(my_hist1): #iki ayrı listeye bölüyoruz, en cok okunan sayfanın hangisi olduğunu ve kaç kere okunduğunu buluyoruz.
     vlist=list(my_hist1.values()) #value listesi
     klist=list(my_hist1.keys()) #key listesi
     return klist[vlist.index(max(vlist))],max(vlist) #burda da valueler arasında maksimumun indexini keylistemize karşılık gelen değere göre yani en çok okunan sayfayı buluyoruz.
#max(vlist)' te ise kaç kere okunduğunu döndürüyor


In [5]:
#def genPowerset(iterable): #olabilecek bütün koşulları kombine etmeye yarıyor. Kendim yazdığım için yorum satırına aldım
#    "powerset([1,2,3])--> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)" #Bütün olası değerleri kombine eder.
#    s=list(iterable)
#    return chain.from_iterable(combinations(s,r)for r in range(len(s)+1))


In [6]:
#pset : [(10,20),(15,25),(30,50),(40,60),(35,40)]  best list = {(10,20),(35,40),(40,60)} #öylesine örnek yazdım
def chooseBest(pset, getbasSayfa, getsonSayfa):   #Brute force yöntemi için yazılan fonksiyonumuz.
    bestSayfa=0 
    bestSet=None #en iyi seti anlamak için pset içindeki
    for items in pset:  #ilk başta itemslerimiz pset içinde olan listelerde geziyor.
       #for item in items:
            #print(item)
        itemsSayfa=0
        durum = controlKesisen(items,Item.getbasSayfa,Item.getsonSayfa) #eğer pset içindeki listelerin içindeki itemler arasında keşisen var ise False döner. Ve bu şekilde koşula girmemeyi sağlıyoruz.
        if(durum): #kesisen eleman var ise hic girmez
            for item in items:
                itemsSayfa+=getsonSayfa(item)-getbasSayfa(item) #Ortak olmayan elemanlardaki maksimum sayfa sayısını bulmak için böyle yapıyoruz.
            if itemsSayfa >= bestSayfa: #psetteki maksimum okunan sayfayı bulmak maksadımız
                bestSayfa=itemsSayfa
                bestSet=items
    return (bestSet,bestSayfa)


In [7]:
def chooseBestMemo(pset, getbasSayfa, getsonSayfa,memo={}):  
    bestSayfa=0 
    bestSet=None #en iyi seti anlamak için pset içindeki
    for items in pset: 
        #for item in items:
            #print("itemlerimiz:",item)
        itemsSayfa=0
        durum = controlKesisen(items,Item.getbasSayfa,Item.getsonSayfa)  #eğer pset içindeki listelerin içindeki itemler arasında keşisen var ise False döner. Ve bu şekilde koşula girmemeyi sağlıyoruz.
        if(durum): #kesisen eleman var ise hic girmez
            for item in items:
                if (item in memo.keys()): #en altta uzun uzun anlattım. Amacımız eğer bellek içinde itemSayfa var ise direk bellekten çekerek fonksiyonumuzun daha hızlı çalışmasını sağlamak diyebiliriz.
                    memosayfa=memo[item]
                    itemsSayfa+=memosayfa

                else: 
                     memosayfa=getsonSayfa(item)-getbasSayfa(item) #Burda bunu ayrı yazmamızın sebebi benim amacım pset içindeki listelerin içindeki her iteme karşılık gelen değeri bulmak. Onun için bunu ayrı yazmak durumunda oluyorum.
                     itemsSayfa+=getsonSayfa(item)-getbasSayfa(item) #Ortak olmayan elemanlardaki maksimum sayfa sayısını bulmak için böyle yapıyoruz. 
                     memo[item]=memosayfa #burada eğer itemımız memo da yok ise ekliyoruz ama bunu memosayfaya göre yapmamız gerek eğer itemsayfaya göre yaparsak psetin içindeki listeler için yapmış oluruz.Ama bizim amacımız psetin içindeki listelerin içindeki itemlere göre yapmak.
                     memosayfa=0  
        if itemsSayfa >= bestSayfa: #psetteki maksimum okunan sayfayı bulmak maksadımız 
                 bestSayfa=itemsSayfa
                 bestSet=items
                 
    
    return (bestSet,bestSayfa)

In [8]:
def controlKesisen(items,getbasSayfa, getsonSayfa):
    sonuc=True #eğer kesisen eleman yok ise sonucu direk True olarak dönderiyoruz. 
#items: [(10,20),(15,25),(30,50),(40,60),(35,40)] bestset=[(10,20),(30,50),(40,60)   Örnek olarak yazmıstım

    n=len(items)      
    for i in range(0,n-1,1): #şimdi yukardaki örnekten hareket ederek listemizde bulunan itemlerin herbirini birini kendinden sonraki gelenler ile kıyaslayarak kesişip kesişmediğini buluyoruz.
        for j in range(i,n-1,1): 
            #bütün listede kesisen elemanlari bulmamızı sağlar,eğer bu durumlardan herhangi biri sağlanırsa kesişen item vardır. Ve sonucumuz False döner.
            if((getbasSayfa(items[i])>= getbasSayfa(items[j+1]) and getbasSayfa(items[i]) <= getsonSayfa(items[j+1]) ) or(getbasSayfa(items[j+1])>=getbasSayfa(items[i]) and getbasSayfa(items[j+1])<= getsonSayfa(items[i])) or(getbasSayfa(items[i])<=getbasSayfa(items[j+1]) and getsonSayfa(items[j+1]) <= getsonSayfa(items[i])) or(getbasSayfa(items[j+1])<=getbasSayfa(items[i]) and getsonSayfa(items[i]) <= getsonSayfa(items[j+1]))):
                sonuc=False #yani kesisiyor
                break   
    
    return sonuc

In [9]:
def myPset(items,getbasSayfa, getsonSayfa): #kendi oluşturduğum pset, böylece karmaşıklık minimuma indirmiş oldum. Bu psetin icinde sadece birbiri ile kesişmeyen ayrı ayrı listeleri tutuyorum.
    iclist=[]
    n=len(items)
    pset= [[]] #kendi olusturacağımız psetimiz
    for i in range(0,n-1,1):
        iclist.append(items[i])
        for j in range(i,n-1,1): 
            #eğer bu durumlardan biri sağlanıyor ise kesişiyor demektir listedeki elemanımız onun için oluşturduğumuz listeye ekleme yapmıyoruz o itemsı.
            if((getbasSayfa(items[i])>= getbasSayfa(items[j+1]) and getbasSayfa(items[i]) <= getsonSayfa(items[j+1]) ) or(getbasSayfa(items[j+1])>=getbasSayfa(items[i]) and getbasSayfa(items[j+1])<= getsonSayfa(items[i])) or(getbasSayfa(items[i])<=getbasSayfa(items[j+1]) and getsonSayfa(items[j+1]) <= getsonSayfa(items[i])) or(getbasSayfa(items[j+1])<=getbasSayfa(items[i]) and getsonSayfa(items[i]) <= getsonSayfa(items[j+1]))):                
                break   
            else: #eğer kesişmiyor ise ekleme yapıyoruz listemize.
                iclist.append(items[j+1])

        pset.append(iclist); #iç içe bir çok birbiri ile kesişmeyen liste elde ediyoruz. Bunun sayesinde kesişen itemleri içinde bulundurmadığı için karmaşıklık minimalize etmiş olduk.
        iclist=[] #bu sekilde ekledikten sonra listemizi tekrardan sıfırlıyoruz. en baştan yeni bir liste psetin içine ekliyoruz.
    return pset;

In [10]:
def BruteForceTest(items,getbasSayfa,getsonSayfa): #print işlemleri için yazdık Brute Forcenin
    pset=myPset(items,Item.getbasSayfa,Item.getsonSayfa) #karmaşıklık 0(n) iken genPoweset yüzünden 0(n*2^n) oluyor. || pset: Olası bütün durumlar.
    taken,val= chooseBest(pset,Item.getbasSayfa,Item.getsonSayfa)
    print('Maks okunan sayfa:',val)
    for item in taken:
        print(item)

In [11]:
def DynamicProgrammingTest(items,getbasSayfa, getsonSayfa): #Karmaşıklığı az olan Memodan yardım alarak yazdıgım fonksiyonun print işlemleri.
    pset=myPset(items,Item.getbasSayfa,Item.getsonSayfa) #karmaşıklık 0(n) iken genPoweset yüzünden 0(n*2^n) oluyor. || pset: Olası bütün durumlar.
    taken,val= chooseBestMemo(pset,Item.getbasSayfa,Item.getsonSayfa)
    print('Maks okunan sayfa:',val)
    for item in taken:
        print(item)

In [12]:
try: #bu sayede int deger dısında birşey girersek default değer olarak kisiS=5,sayfaS=250 alıyoruz.
    kisiS=int(input("kisi sayisi giriniz:"))
    sayfaS=int(input("sayfa sayisi giriniz:"))
except ValueError:
    kisiS=5
    sayfaS=250

kisi sayisi giriniz:10
sayfa sayisi giriniz:200


In [13]:
items,attachedList=randList(kisiS,sayfaS) 

print("Üretilen İtemlerimiz:") #yani ilk ve son sayfa
for item in items:
    print(item)

print("----------------------------------------------------------------------------------------")
print()
#print(attachedList)

my_hist1=get_hist(attachedList)
mostreadpage,readNumberOfPage=mostReadPage(my_hist1)

print("<","En cok okunan sayfa:",mostreadpage," ","Okundugu sayfa sayisi:",readNumberOfPage,">")
print()
print("Ortak okuduğu sayfa olmayanlar:")
print("BruteForce ile yaptığım:")
BruteForceTest(items,Item.getbasSayfa,Item.getsonSayfa)
print()
print("Dynamic Programming ile yaptığım")
DynamicProgrammingTest(items,Item.getbasSayfa,Item.getsonSayfa)

Üretilen İtemlerimiz:
<36, 68>
<107, 160>
<150, 164>
<11, 32>
<90, 114>
<91, 116>
<85, 111>
<44, 156>
<76, 200>
<42, 80>
----------------------------------------------------------------------------------------

< En cok okunan sayfa: 107   Okundugu sayfa sayisi: 6 >

Ortak okuduğu sayfa olmayanlar:
BruteForce ile yaptığım:
Maks okunan sayfa: 124
<76, 200>

Dynamic Programming ile yaptığım
Maks okunan sayfa: 124
<76, 200>


In [ ]:
"""
A-)Bu listeyi random olarak üreten fonksiyonu yazınız. Kişi sayısı ( default değer 5,) ve kitabın sayfa sayısı 
( default değer 250 ) fonksiyona parametre olarak gönderilmelidir. (Cevap)
    Bunu bulmak içinde kullanıcıdan önce bir kişi sayısı ve kitabın sayfa sayısını istedim. Sonra bunları randList fonksiyonumuza
atadım. Bu fonksiyonda random bir şekilde kitabın son ve bas sayfasını bulduk. Ve sonra bunu items adlı listemize Item classımız
yardımı ile ekledim. Kisi sayısına göre bunu durmadan tekrar ettirdim. 

B-) En çok okunan sayfayı bulunuz.(Cevap)
    Bunu bulmak için yaptığım şey ilk önce bütün listemizdeki itemleri tektek ilk sayfa ve son sayfalarına göre açtım. 
Bunların sonra bir listede tutup histogram fonksiyonuna yolladım.Sonra bu histogramın aralarındaki en yüksek value değerine 
göre yani en çok okunan sayfayı bulmak için mostReadPage fonksiyonumuza histogramı atadım ve en çok okunan sayfa ve okunduğu 
sayfa sayısını bulduk.

C.)    Bu listede öyle k tane kişiden oluşan öyle bir grup vardır ki bunların ortak okuduğu sayfa yoktur ve 
birleşimi okunan sayfa sayısı olarak maksimumdur. Bu k kişiden oluşan grubu bulunuz. ( Brute Force yöntemi kullanmayınız) (Cevap)
    İlk önceklikle karmaşıklığı minimuma indirmek için myPset adlı fonksiyon yazdım. Bu fonksiyon iç içe listelere sahip pset 
ve iclist adlı değişkene sahip. Burdaki pseti oluştururken amacım itemslerimizde olan değişkenlerin kesişmeyenlerinin oluştuğu
bir pset yapmak. Bunuda iclist yardımıyla ilk başta ilk elemanımızı ekleyip ondan sonra diğer elemanları kapsamıyor ise yani
ortak eleman yok ise bunu listeye ekliyoruz. Bu şekilde dış döngüdeki durmadan i değişkeni değiştikce psetimiz içinde yeni 
bir liste oluşmuş oluyor. Bu şekilde sadece psetimiz içinde kesişmeyen elemanların olduğu listeler olmuş oluyor.
Daha sonra bunu oluşturmuş olduğum fonksiyonum olan chooseBestMemo fonksiyonumuza yolluyorum. Bu fonksiyonda geriye best sayfa
ve best set döndermesini istiyoruz. Burda amacımız memo yardımı ile itemlerimizin farkı olan itemsSayfa'yı bulurken eğer önceden
bulduk isek tekrar bunu hesaplamak yerine bu değeri bellekten çekmemiz. Bu şekilde yapmamız ile programımızı hızlandırmanın yanı
sıra karmaşıklık içinde minimalize daha çok yaklaşmış oluyoruz. Örneğin;
liste=[(100,20),(200,100),(300,100),(400,100),(500,300)] elimizde böyle bir liste var. Memomuza ekliceğimiz olan değerler
bu listedeki son eleman ile ilk elemanın farkıdır. Bu şekilde psetimiz içindeki listemizde tekrar böyle itemler ile karşılaşılır
ise direk farklarını yani okuduğu sayfayı döndürerek hesaplamaya gerek bırakmamaktır. Memomuzda mesela bu şekil olmuş olur 
örneğimize göre;
memo={((100,20):80),((200,100):100),((300,100):200),((400,100):300),((500,300):200)}
Bundan önce birde controlKesisen adlı fonksiyonumuz yardımı ile  pset içinde olan listelerde kesişen eleman olup olmadığını 
anlıyoruz. Eğer kesişen eleman var ise False döner, yoksa True döner. Bunuda anlamak için bu fonksiyonumuzda Listemiz içinde
olan itemlerin bas sayfa ve son sayfalarını itemler arasında kıyaslayarak itemlerin ortak elemanı olup olmadığını anlamaya yarıyor
eğer ortak eleman bulur ise yani kesişme şartlarımız için listemizde bulunan itemlerimiz if'in içindeki herhangi bir durumu 
sağlar ise bunların kesiştiğini anlayıp  false döndürüryoruz ve diğer fonksiyonumuz içindeki psette  bulunan listede kesişen
eleman olduğu için o liste için girmeyip koşulumuza bir sonraki liste için bunu tekrarlıyoruz. chooseBestMemo fonksiyonumuz 
içinde bulunan en son if koşulumuz ile en best olan sayfa sayesinde best set ve bestSayfamızı bulmuş oluyoruz.

D.)Brute Force yöntemi nedir? Yukarıdaki C seçeneğini brute force yöntemi ile bulunuz. (Cevap)
   Brute Force yöntemi kaba kuvvet diye de bildiğimiz sırt çanta problemlerinin çözüm yöntemi diyebiliriz. Bu sırt çanta problem
lerinin tabi ki durumları değişebilir. Bu kodumuzdaki durum ise ortak olmayan şekilde okunan sayfa sayılarına göre maksimum
okunan sayfayı bulmaktır. Optimal (olası en iyi durum) bulmak için uğraşırız. Brute forcee yönteminin izlediği adımda ise
döngü yardımıyla olasılık durumlarını test etmekten geçer. Bu bütün olasılık durumlarına ise power set deriz.
Hatta bunun izlediği algoritmaya şöyle diyebiliriz:
1.Adım: Hepsini Üret
2.Adım: Şartı sağlamayanları çıkart.
3.Adım: Şartı sağlayanlardan maksimum olanları geri dönder. 
Kitabımızda bulunan chooseBest fonksiyonumuzda bu işe yaramaktadır. 

    Birde karmaşıklıktan bahsetmek istiyorum benim amacım yazdığım kodda karmaşıklığı minimuma indirmeye çalışmaktır. 
Bunuda minimuma indirmek için power set’i yani olası tüm durumları minimalize ederek yapmaya çalıştım. 
Önce power set’in karmaşıklığından bahsedecek olursam eğer O(2nxn)’dir bunu ben azaltmak için tüm durumlar 
yerine kesişmeyenleri tutan bir power set elde ederek karmaşıklığımı minimuma indirmeye çalıştım. Birde buna 
ekstradan kitapta da geçen memoization yönteminden yardım aldım. Bunuda bellekde eğer itemsımızın sayfa sayısı 
var ise burdan çekerek yaptım. Bu şekilde karmaşıklık için en iyi şekilde kod yazmış oldum. 

    
""""